In [1]:
# meta-data does not work yet in VScode
# https://github.com/microsoft/vscode-jupyter/issues/1121

{
    "tags": [
        "hide-cell"
    ]
}


### Install necessary libraries

try:
    import jax
except:
    # For cuda version, see https://github.com/google/jax#installation
    %pip install --upgrade "jax[cpu]" 
    import jax

try:
    import distrax
except:
    %pip install --upgrade  distrax
    import distrax

try:
    import jsl
except:
    %pip install git+https://github.com/probml/jsl
    import jsl

try:
    import rich
except:
    %pip install rich
    import rich




     |████████████████████████████████| 272 kB 6.9 MB/s eta 0:00:01
  Using cached tensorflow_probability-0.16.0-py2.py3-none-any.whl (6.3 MB)
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-probability 0.13.0
    Uninstalling tensorflow-probability-0.13.0:
      Successfully uninstalled tensorflow-probability-0.13.0
  Attempting uninstall: distrax
    Found existing installation: distrax 0.0.1
    Uninstalling distrax-0.0.1:
      Successfully uninstalled distrax-0.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jsl 0.0.0 requires dataclasses, which is not installed.
You should consider upgrading via the '/opt/anaconda3/envs/spyder-dev/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
{
    "tags": [
        "hide-cell"
    ]
}


### Import standard libraries

import abc
from dataclasses import dataclass
import functools
import itertools

from typing import Any, Callable, NamedTuple, Optional, Union, Tuple

import matplotlib.pyplot as plt
import numpy as np


import jax
import jax.numpy as jnp
from jax import lax, vmap, jit, grad
from jax.scipy.special import logit
from jax.nn import softmax
from functools import partial
from jax.random import PRNGKey, split

import inspect
import inspect as py_inspect
import rich
from rich import inspect as r_inspect
from rich import print as r_print

def print_source(fname):
    r_print(py_inspect.getsource(fname))

(sec:nlds-intro)=
# Nonlinear Gaussian SSMs

In this section, we consider SSMs in which the dynamics and/or observation models are nonlinear,
but the process noise and observation noise are Gaussian.
That is, 
\begin{align}
\hmmhid_t &= \ssmDynFn(\hmmhid_{t-1}, \inputs_t) +  \vepsilon_t  \\
\hmmobs_t &= \ssmObsFn(\hmmhid_{t}, \inputs_t) + \veta_t
\end{align}
where $\vepsilon_t \sim \gauss(\vzero,\vQ_t)$
and $\veta_t \sim \gauss(\vzero,\vR_t)$.
This is a very widely used model class. We give some examples below.

(sec:pendulum)=
## Example: tracking a 1d pendulum

```{figure} /figures/pendulum.png
:scale: 50%
:name: fig:pendulum

Illustration of a pendulum swinging.
$g$ is the force of gravity,
$w(t)$ is a random external force,
and $\alpha$ is the angle wrt the vertical.
Based on {cite}`Sarkka13` fig 3.10.
```


% Sarka p45, p74
Consider a simple pendulum of unit mass and length swinging from
a fixed attachment, as in
{numref}`Figure %s <fig:pendulum>`.
Such an object is in principle entirely deterministic in its behavior.
However, in the real world, there are often unknown forces at work
(e.g., air turbulence, friction).
We will model these by a continuous time random Gaussian noise process $w(t)$.
This gives rise to the following differential equation:
\begin{align}
\frac{d^2 \alpha}{d t^2}
= -g \sin(\alpha) + w(t)
\end{align}
We can write this as a nonlinear SSM by defining the state to be
$z_1(t) = \alpha(t)$ and $z_2(t) = d\alpha(t)/dt$.
Thus
\begin{align}
\frac{d \vz}{dt}
= \begin{pmatrix} z_2 \\ -g \sin(z_1) \end{pmatrix}
+ \begin{pmatrix} 0 \\ 1 \end{pmatrix} w(t)
\end{align}
If we discretize this step size $\Delta$,
we get the following
formulation {cite}`Sarkka13` p74:
\begin{align}
\underbrace{
  \begin{pmatrix} z_{1,t} \\ z_{2,t} \end{pmatrix}
  }_{\hmmhid_t}
=
\underbrace{
  \begin{pmatrix} z_{1,t-1} + z_{2,t-1} \Delta  \\
    z_{2,t-1} -g \sin(z_{1,t-1}) \Delta  \end{pmatrix}
  }_{\vf(\hmmhid_{t-1})}
+\vq_{t-1}
\end{align}
where $\vq_{t-1} \sim \gauss(\vzero,\vQ)$ with
\begin{align}
\vQ = q^c \begin{pmatrix}
  \frac{\Delta^3}{3} &   \frac{\Delta^2}{2} \\
  \frac{\Delta^2}{2} & \Delta
  \end{pmatrix}
  \end{align}
where $q^c$ is the spectral density (continuous time variance)
of the continuous-time noise process.


If we observe the angular position, we
get the linear observation model
\begin{align}
y_t = \alpha_t + r_t =  h(\hmmhid_t) + r_t
\end{align}
where $h(\hmmhid_t) = z_{1,t}$
and $r_t$ is the observation noise.
If we only observe  the horizontal position,
we get the nonlinear observation model
\begin{align}
y_t = \sin(\alpha_t) + r_t =  h(\hmmhid_t) + r_t
\end{align}
where $h(\hmmhid_t) = \sin(z_{1,t})$.





